<a href="https://colab.research.google.com/github/raheeltahir55/CE888/blob/main/Project/Proj_MultiEval_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
### EMOTION ####
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import io
import requests

In [89]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import SpatialDropout1D
import tensorflow as tf

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# from google.colab import files
# uploaded= files.upload()
# file= "Train_Text.txt"
# uploaded[file].decode("utf-8").split("\r\n")

r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt', allow_redirects= True)
open('train_text.txt', 'wb').write(r.content)
example1 = "/content/train_text.txt"
with open(example1, "r") as file1:
    data = file1.readlines()

In [5]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt', allow_redirects= True)
open('train_labels.txt', 'wb').write(r.content)
example1 = "/content/train_labels.txt"
with open(example1, "r") as file1:
    data1 = file1.readlines()

In [6]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt', allow_redirects= True)
open('test_text.txt', 'wb').write(r.content)
example1 = "/content/test_text.txt"
with open(example1, "r") as file1:
    data2 = file1.readlines()

In [7]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt', allow_redirects= True)
open('test_labels.txt', 'wb').write(r.content)
example1 = "/content/test_labels.txt"
with open(example1, "r") as file1:
    data3 = file1.readlines()

In [8]:
# data = uploaded[file].decode("utf-8").split("\r\n")
# data1 = uploaded1[file1].decode("utf-8").split("\r\n")
# data2 = uploaded2[file2].decode("utf-8").split("\r\n")
# data3 = uploaded3[file3].decode("utf-8").split("\r\n")

data= [data[i].strip() for i in range(len(data))]
data1= [data1[i].strip() for i in range(len(data1))]
data2= [data2[i].strip() for i in range(len(data2))]
data3= [data3[i].strip() for i in range(len(data3))]

In [49]:
### df is dataframe for training data

df= pd.DataFrame()
for index in range(len(data)):
  df.loc[index, 'Text']=data[index]
df.head()

,Text
0,“Worry is a down payment on a problem you may ...
1,My roommate: it's okay that we can't spell bec...
2,No but that's so cute. Atsu was probably shy a...
3,Rooneys fucking untouchable isn't he? Been fuc...
4,it's pretty depressing when u hit pan on ur fa...


In [50]:
for index in range(len(data1)):
  df.loc[index, 'Label']=data1[index]
df.head()

,Text,Label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [51]:
### df1 is dataframe for test data

df1= pd.DataFrame()
for index in range(len(data2)):
  df1.loc[index, 'Text']=data2[index]
df1.head()

,Text
0,#Deppression is real. Partners w/ #depressed p...
1,@user Interesting choice of words... Are you c...
2,My visit to hospital for care triggered #traum...
3,@user Welcome to #MPSVT! We are delighted to h...
4,What makes you feel #joyful?


In [52]:
for index in range(len(data3)):
  df1.loc[index, 'Label']=data3[index]
df1.head()

,Text,Label
0,#Deppression is real. Partners w/ #depressed p...,3
1,@user Interesting choice of words... Are you c...,0
2,My visit to hospital for care triggered #traum...,3
3,@user Welcome to #MPSVT! We are delighted to h...,1
4,What makes you feel #joyful?,1


In [54]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [55]:
# Step - a : Remove blank rows if any.

#df['Text']= df1['Text'].astype(str)

df['Text'].dropna(inplace=True)

# Step - b : Change all the text to lower case.
df['Text'] = [entry.lower() for entry in df['Text']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df['Text']= [word_tokenize(entry) for entry in df['Text']]

for index,entry in enumerate(df['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'Text_Final'] = str(Final_words) 

In [56]:
# Step - a : Remove blank rows if any.

#df1['Text']= df1['Text'].astype(str)

df1['Text'].dropna(inplace=True)

# Step - b : Change all the text to lower case.
df1['Text'] = [entry.lower() for entry in df1['Text']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df1['Text']= [word_tokenize(entry) for entry in df1['Text']]

for index,entry in enumerate(df1['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df1.loc[index,'Text_Final'] = str(Final_words) 

In [57]:
t_df= pd.concat([df, df1])
t_df['Label'].value_counts()

0    1958
3    1237
1    1066
2     417
Name: Label, dtype: int64

In [58]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 10000
# Max number of words in each tweet.
MAX_SEQUENCE_LENGTH = 18
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(t_df['Text_Final'].values)
word_index = tokenizer.word_index

In [59]:
print('Found %s unique tokens.' % len(word_index))

Found 8544 unique tokens.


In [60]:
print(word_index)

{"'user'": 1, "'get'": 2, "'like'": 3, "'go'": 4, "'amp'": 5, "'make'": 6, "'u'": 7, "'people'": 8, "'one'": 9, "'know'": 10, "'think'": 11, "'love'": 12, "'say'": 13, "'day'": 14, "'time'": 15, "'look'": 16, "'see'": 17, "'feel'": 18, "'want'": 19, "'sad'": 20, "'need'": 21, "'would'": 22, "'really'": 23, "'good'": 24, "'bad'": 25, "'watch'": 26, "'even'": 27, "'still'": 28, "'take'": 29, "'back'": 30, "'ca'": 31, "'fuck'": 32, "'work'": 33, "'thing'": 34, "'today'": 35, "'fear'": 36, "'lose'": 37, "'life'": 38, "'start'": 39, "'never'": 40, "'let'": 41, "'someone'": 42, "'angry'": 43, "'much'": 44, "'come'": 45, "'try'": 46, "'happy'": 47, "'depression'": 48, "'year'": 49, "'way'": 50, "'right'": 51, "'well'": 52, "'new'": 53, "'tell'": 54, "'use'": 55, "'man'": 56, "'sadness'": 57, "'could'": 58, "'play'": 59, "'terrorism'": 60, "'stop'": 61, "'rage'": 62, "'anger'": 63, "'find'": 64, "'awful'": 65, "'ever'": 66, "'im'": 67, "'give'": 68, "'fume'": 69, "'night'": 70, "'anxiety'": 71

In [61]:
df.head()

,Text,Label,Text_Final
0,"[“, worry, is, a, down, payment, on, a, proble...",2,"['worry', 'payment', 'problem', 'may', 'never'..."
1,"[my, roommate, :, it, 's, okay, that, we, ca, ...",0,"['roommate', 'okay', 'ca', 'spell', 'autocorre..."
2,"[no, but, that, 's, so, cute, ., atsu, was, pr...",1,"['cute', 'atsu', 'probably', 'shy', 'photo', '..."
3,"[rooneys, fucking, untouchable, is, n't, he, ?...",0,"['rooneys', 'fuck', 'untouchable', 'fuck', 'dr..."
4,"[it, 's, pretty, depressing, when, u, hit, pan...",3,"['pretty', 'depress', 'u', 'hit', 'pan', 'ur',..."


In [62]:
df1.head()

,Text,Label,Text_Final
0,"[#, deppression, is, real, ., partners, w/, #,...",3,"['deppression', 'real', 'partner', 'depressed'..."
1,"[@, user, interesting, choice, of, words, ...,...",0,"['user', 'interesting', 'choice', 'word', 'con..."
2,"[my, visit, to, hospital, for, care, triggered...",3,"['visit', 'hospital', 'care', 'trigger', 'trau..."
3,"[@, user, welcome, to, #, mpsvt, !, we, are, d...",1,"['user', 'welcome', 'mpsvt', 'delight', 'grate..."
4,"[what, makes, you, feel, #, joyful, ?]",1,"['make', 'feel', 'joyful']"


In [63]:
X_train = tokenizer.texts_to_sequences(df['Text_Final'].values)
X_test = tokenizer.texts_to_sequences(df1['Text_Final'].values)

In [64]:
X_train[3], X_train[1], X_test[1], X_test[3]

([3738, 32, 3739, 32, 282, 3740, 16, 1427, 3741, 148],
 [1425, 349, 31, 1776, 1426, 88, 2368],
 [1, 1290, 802, 146, 3232, 520, 1750, 60, 191, 432, 548, 28],
 [1, 985, 3578, 417, 1201, 3578, 624])

In [65]:
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (3257, 18)
Shape of data tensor: (1421, 18)


In [66]:
X_train[1,:], X_test[1,:]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        1425,  349,   31, 1776, 1426,   88, 2368], dtype=int32),
 array([   0,    0,    0,    0,    0,    0,    1, 1290,  802,  146, 3232,
         520, 1750,   60,  191,  432,  548,   28], dtype=int32))

In [67]:
X_train[:,3], X_test[:,3]

(array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 array([160,   0,   0, ...,   0,   0,   0], dtype=int32))

In [68]:
df= pd.get_dummies(df, columns= ['Label'])

In [69]:
out_test= df1['Label'].values
df1= pd.get_dummies(df1, columns= ['Label'])

In [70]:
df.head()

,Text,Text_Final,Label_0,Label_1,Label_2,Label_3
0,"[“, worry, is, a, down, payment, on, a, proble...","['worry', 'payment', 'problem', 'may', 'never'...",0,0,1,0
1,"[my, roommate, :, it, 's, okay, that, we, ca, ...","['roommate', 'okay', 'ca', 'spell', 'autocorre...",1,0,0,0
2,"[no, but, that, 's, so, cute, ., atsu, was, pr...","['cute', 'atsu', 'probably', 'shy', 'photo', '...",0,1,0,0
3,"[rooneys, fucking, untouchable, is, n't, he, ?...","['rooneys', 'fuck', 'untouchable', 'fuck', 'dr...",1,0,0,0
4,"[it, 's, pretty, depressing, when, u, hit, pan...","['pretty', 'depress', 'u', 'hit', 'pan', 'ur',...",0,0,0,1


In [71]:
df1.head()

,Text,Text_Final,Label_0,Label_1,Label_2,Label_3
0,"[#, deppression, is, real, ., partners, w/, #,...","['deppression', 'real', 'partner', 'depressed'...",0,0,0,1
1,"[@, user, interesting, choice, of, words, ...,...","['user', 'interesting', 'choice', 'word', 'con...",1,0,0,0
2,"[my, visit, to, hospital, for, care, triggered...","['visit', 'hospital', 'care', 'trigger', 'trau...",0,0,0,1
3,"[@, user, welcome, to, #, mpsvt, !, we, are, d...","['user', 'welcome', 'mpsvt', 'delight', 'grate...",0,1,0,0
4,"[what, makes, you, feel, #, joyful, ?]","['make', 'feel', 'joyful']",0,1,0,0


In [72]:
Labels_=['Label_0','Label_1','Label_2','Label_3']

In [73]:
y_train= df[Labels_].values
y_test= df1[Labels_].values

In [74]:
print(y_train[0,:])

[0 0 1 0]


In [75]:
print(y_train[0:10])

[[0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [112]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)


Epoch 1/10
46/46 [==============================] - 11s 130ms/step - loss: 1.3041 - accuracy: 0.4236 - val_loss: 1.2423 - val_accuracy: 0.4202
Epoch 2/10
46/46 [==============================] - 5s 119ms/step - loss: 1.1076 - accuracy: 0.5250 - val_loss: 0.9680 - val_accuracy: 0.6564
Epoch 3/10
46/46 [==============================] - 5s 118ms/step - loss: 0.5695 - accuracy: 0.7912 - val_loss: 0.8360 - val_accuracy: 0.6994
Epoch 4/10
46/46 [==============================] - 5s 117ms/step - loss: 0.2390 - accuracy: 0.9323 - val_loss: 1.0294 - val_accuracy: 0.6810
Epoch 5/10
46/46 [==============================] - 5s 116ms/step - loss: 0.1136 - accuracy: 0.9686 - val_loss: 1.1758 - val_accuracy: 0.6748
Epoch 6/10
46/46 [==============================] - 5s 117ms/step - loss: 0.0921 - accuracy: 0.9744 - val_loss: 1.2524 - val_accuracy: 0.6748
Epoch 7/10
46/46 [==============================] - 5s 116ms/step - loss: 0.0596 - accuracy: 0.9789 - val_loss: 1.2977 - val_accuracy: 0.6840
Epoch

In [106]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

45/45 [==============================] - 1s 11ms/step - loss: 1.7504 - accuracy: 0.6545
Test set
  Loss: 1.750
  Accuracy: 0.654


In [107]:
y_pred = model.predict(X_test)

In [108]:
print(y_pred[1,:])

[0.9803761  0.00407825 0.00201478 0.01353079]


In [109]:
out_pred= [np.argmax(y_pred[i,:]) for i in range(y_pred.shape[0])]

In [110]:
out_test=out_test.astype(np.int32)
out_pred= np.array(out_pred)
print(out_pred[1:10]), print(out_test[1:10])

[0 3 1 1 3 3 3 0 0]
[0 3 1 1 0 3 3 3 0]


(None, None)

In [111]:
from sklearn.metrics import f1_score
print("F1 Score -> ",f1_score(out_pred, out_test, average='macro')*100)

F1 Score ->  57.84827034078128
